In [1]:
import pyedflib

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import glob
import os
import re

In [ ]:
f = pyedflib.EdfReader('../00_Data/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_01.edf')

In [ ]:
f.getSignalLabels()

In [ ]:
type(f.getSampleFrequencies()[0])

In [ ]:
f.getNSamples()[0]

In [ ]:
signal_labels = f.getSignalLabels()
signal_data = np.zeros((f.getNSamples()[0], len(signal_labels)))
for i, channel in enumerate(signal_labels):
    signal_data[:, i] = f.readSignal(signal_labels.index(channel))

In [ ]:
df = pd.DataFrame(signal_data, columns=signal_labels).astype('float32')

In [ ]:
display(df)

In [ ]:
f1 = go.Figure(
    data = [
        go.Scatter(y=df["FP1-F3"], name="RUL"),
        go.Scatter(y=df["F7-T7"], name="Predicted RUL"),
    ],
    layout = {"yaxis": {"title": "RUL [km]"}, "xaxis": {"title": "Samples"}, "title": "Predicted RUL vs real RUL - TRAIN"}
)

f1.show()

In [2]:
root_path = '../00_Data/chb-mit-scalp-eeg-database-1.0.0/'
all_patients = sorted([patient for patient in os.listdir(root_path) if re.match(r'(chb)\d+', patient)])

In [7]:
patient_dict = {}
info_file = open(root_path + all_patients[0] + '/' + all_patients[0] + '-summary.txt','r').readlines()

In [12]:
patient_dict['Seizures Window'] = []
for line in info_file:
    if(re.findall('Seizure Start Time|Seizure End Time|Seizure \d+ Start Time|Seizure \d+ End Time', line)):
      patient_dict['Seizures Window'].append(int(re.findall('\d+', line)[-1]))
    

In [13]:
patient_dict

{'Start Time': '',
 'Seizures Window': [2996,
  3036,
  1467,
  1494,
  1732,
  1772,
  1015,
  1066,
  1720,
  1810,
  327,
  420,
  1862,
  1963]}

In [ ]:
#re.findall('Seizure Start Time|Seizure End Time|Seizure \d+ Start Time|Seizure \d+ End Time', info_file)
patient_dict['Seizures Window'] = []
patient_dict['Seizures Window'].append(int(re.findall('\d+', info_file)))

In [ ]:
patient_dict

In [ ]:
def read_patient_info(patient:str, root_path:str = '../00_Data/chb-mit-scalp-eeg-database-1.0.0/'):
    patient_dict = {}
    info_file = open(root_path + patient + '/' + patient + '-summary.txt','r')

In [3]:
def get_complete_patient_eeg_data(patient:str, patient_dict:dict, root_path:str = '../00_Data/chb-mit-scalp-eeg-database-1.0.0/', channels:list = []):
    parent_path = root_path + patient
    all_files = glob.glob(os.path.join(parent_path , ("*.edf")))
    print("Test")
    if(len(channels)==0):
        channels = pyedflib.EdfReader(all_files[0]).getSignalLabels()
    concat_list = []
    for chunk in all_files:
        edf_file = pyedflib.EdfReader(chunk)
        signal_data = np.zeros((edf_file.getNSamples()[0], len(channels)))
        for i, channel in enumerate(channels):
            signal_data[:, i] = edf_file.readSignal(channels.index(channel))
        concat_list.append(pd.DataFrame(signal_data, columns=channels).astype('float32'))
    dataframe = pd.concat(concat_list, axis=0, ignore_index=True)
    dataframe["patient"] = patient
    dataframe["timestamp"] = pd.date_range('1970-01-01 00:00:00', freq='3906250N', periods=len(dataframe))
    dataframe["seconds"] = np.floor(np.linspace(0, len(dataframe)/edf_file.getSampleFrequencies()[0], len(dataframe),  endpoint=False)).astype('uint16')
    return dataframe


In [4]:
df = get_complete_patient_eeg_data(all_patients[0], {}, channels=["F4-C4"])

Test


In [5]:
display(df)

,F4-C4,patient,timestamp,seconds
0,50.989010,chb01,1970-01-01 00:00:00.000000000,0
1,0.195360,chb01,1970-01-01 00:00:00.003906250,0
2,0.195360,chb01,1970-01-01 00:00:00.007812500,0
3,0.195360,chb01,1970-01-01 00:00:00.011718750,0
4,-0.195360,chb01,1970-01-01 00:00:00.015625000,0
...,...,...,...,...
37372923,-37.704517,chb01,1970-01-02 16:33:07.980468750,14915
37372924,-52.551891,chb01,1970-01-02 16:33:07.984375000,14915
37372925,-50.207569,chb01,1970-01-02 16:33:07.988281250,14915
37372926,-28.717949,chb01,1970-01-02 16:33:07.992187500,14915


In [ ]:
f.getSampleFrequencies()